#SELECCION Y ETIQUETADO DE LA DATA

In [404]:
import pandas as pd

In [405]:
data=pd.read_csv('Cere Network.csv' , delimiter = ';')


In [406]:
# Eliminar las filas que tengan 0 o NaN en la columna 'marketCap'
data = data.query('marketCap != 0').dropna(subset=['marketCap'])

# Verificar que se hayan eliminado las filas
print("Número de filas después de eliminar:", len(data_clean))

Número de filas después de eliminar: 800


In [407]:
data['circulating-supply'] = data['marketCap']/data['close']
data['timeOpen'] = pd.to_datetime(data['timeOpen'])
data['timeHigh'] = pd.to_datetime(data['timeHigh'])
data['timeLow'] = pd.to_datetime(data['timeLow'])

In [408]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 802 entries, 0 to 801
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   timeOpen            802 non-null    datetime64[ns, UTC]
 1   timeHigh            802 non-null    datetime64[ns, UTC]
 2   timeLow             802 non-null    datetime64[ns, UTC]
 3   name                802 non-null    object             
 4   open                802 non-null    float64            
 5   high                802 non-null    float64            
 6   low                 802 non-null    float64            
 7   close               802 non-null    float64            
 8   volume              802 non-null    float64            
 9   marketCap           802 non-null    float64            
 10  circulating-supply  802 non-null    float64            
dtypes: datetime64[ns, UTC](3), float64(7), object(1)
memory usage: 75.2+ KB


In [409]:
import numpy as np
umbral_capitalizacion = 100000000  # $100 millones
data['marketCap_state'] = data['marketCap'].apply(lambda x: 1 if x >= umbral_capitalizacion else 0)

# Calcular los retornos logarítmicos y la volatilidad diaria
data['volatility'] = data['close'].pct_change().rolling(window=2, min_periods=1).std()
# Determinar si la volatilidad está por encima del percentil 90
data['high_volatility'] = np.where(data['volatility'] > data['volatility'].quantile(0.90), 1, 0)

# Calcular el umbral absoluto del aumento del suministro
umbral_aumento_suministro = 0.05 * data['circulating-supply'][0]  # 5% del suministro inicial
# Determinar si el aumento del suministro supera el umbral
data['supply_increase'] = (data['circulating-supply'].diff() > umbral_aumento_suministro).astype(int)

In [410]:
data = data.iloc[2:].reset_index(drop=True)

In [411]:
data

,timeOpen,timeHigh,timeLow,name,open,high,low,close,volume,marketCap,circulating-supply,marketCap_state,volatility,high_volatility,supply_increase
0,2024-06-08 00:00:00+00:00,2024-06-08 18:39:00+00:00,2024-06-08 07:08:00+00:00,Cere Network,0.005589,0.006322,0.005488,0.006041,802499.30,41923867.46,6.939924e+09,0,0.047827,0,0
1,2024-06-07 00:00:00+00:00,2024-06-07 12:36:00+00:00,2024-06-07 22:49:00+00:00,Cere Network,0.005770,0.006435,0.005577,0.005592,920215.00,38805691.98,6.939923e+09,0,0.066853,0,0
2,2024-06-06 00:00:00+00:00,2024-06-06 18:32:00+00:00,2024-06-06 04:36:00+00:00,Cere Network,0.005132,0.006327,0.005103,0.005769,1187795.97,40034399.26,6.939924e+09,0,0.074982,0,0
3,2024-06-05 00:00:00+00:00,2024-06-05 21:07:00+00:00,2024-06-05 11:13:00+00:00,Cere Network,0.005095,0.005182,0.005006,0.005132,860045.78,35612578.26,6.939923e+09,0,0.100489,1,0
4,2024-06-04 00:00:00+00:00,2024-06-04 00:02:00+00:00,2024-06-04 14:53:00+00:00,Cere Network,0.005160,0.005164,0.005030,0.005091,695805.85,35331886.97,6.939924e+09,0,0.072527,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2022-04-05 00:00:00+00:00,2022-04-05 02:19:00+00:00,2022-04-05 23:58:00+00:00,Cere Network,0.037799,0.038322,0.034613,0.034622,6645413.55,54893868.84,1.585542e+09,0,0.133383,1,0
796,2022-04-04 00:00:00+00:00,2022-04-04 23:05:00+00:00,2022-04-04 15:32:00+00:00,Cere Network,0.036627,0.038402,0.036387,0.037804,6764200.95,59939283.17,1.585542e+09,0,0.057315,0,0
797,2022-04-03 00:00:00+00:00,2022-04-03 04:47:00+00:00,2022-04-03 21:49:00+00:00,Cere Network,0.037763,0.038201,0.036233,0.036640,6452226.63,58094209.23,1.585542e+09,0,0.086758,1,0
798,2022-04-02 00:00:00+00:00,2022-04-02 05:37:00+00:00,2022-04-02 15:55:00+00:00,Cere Network,0.038077,0.039603,0.036431,0.037759,7723944.93,59867811.88,1.585542e+09,0,0.043354,0,0


In [412]:
# Definir la función para etiquetar las predicciones
def etiquetar_prediccion(row):
    if row['marketCap_state'] == 1 and row['high_volatility'] == 0 and (row['supply_increase'] == 0 or pd.isna(row['supply_increase'])):
        return 0  # Aumento en el precio
    elif row['marketCap_state'] == 0 and row['high_volatility'] == 1 and (row['supply_increase'] == 1 or pd.isna(row['supply_increase'])):
        return 1  # Disminución en el precio
    elif row['marketCap_state'] == 1 and row['high_volatility'] == 0 and (row['supply_increase'] == 1 or pd.isna(row['supply_increase'])):
        return 2  # Precio estable
    else:
        return 1

# Aplicar la función para etiquetar las predicciones
data['tags'] = data.apply(etiquetar_prediccion, axis=1)


In [413]:
data

,timeOpen,timeHigh,timeLow,name,open,high,low,close,volume,marketCap,circulating-supply,marketCap_state,volatility,high_volatility,supply_increase,tags
0,2024-06-08 00:00:00+00:00,2024-06-08 18:39:00+00:00,2024-06-08 07:08:00+00:00,Cere Network,0.005589,0.006322,0.005488,0.006041,802499.30,41923867.46,6.939924e+09,0,0.047827,0,0,1
1,2024-06-07 00:00:00+00:00,2024-06-07 12:36:00+00:00,2024-06-07 22:49:00+00:00,Cere Network,0.005770,0.006435,0.005577,0.005592,920215.00,38805691.98,6.939923e+09,0,0.066853,0,0,1
2,2024-06-06 00:00:00+00:00,2024-06-06 18:32:00+00:00,2024-06-06 04:36:00+00:00,Cere Network,0.005132,0.006327,0.005103,0.005769,1187795.97,40034399.26,6.939924e+09,0,0.074982,0,0,1
3,2024-06-05 00:00:00+00:00,2024-06-05 21:07:00+00:00,2024-06-05 11:13:00+00:00,Cere Network,0.005095,0.005182,0.005006,0.005132,860045.78,35612578.26,6.939923e+09,0,0.100489,1,0,1
4,2024-06-04 00:00:00+00:00,2024-06-04 00:02:00+00:00,2024-06-04 14:53:00+00:00,Cere Network,0.005160,0.005164,0.005030,0.005091,695805.85,35331886.97,6.939924e+09,0,0.072527,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,2022-04-05 00:00:00+00:00,2022-04-05 02:19:00+00:00,2022-04-05 23:58:00+00:00,Cere Network,0.037799,0.038322,0.034613,0.034622,6645413.55,54893868.84,1.585542e+09,0,0.133383,1,0,1
796,2022-04-04 00:00:00+00:00,2022-04-04 23:05:00+00:00,2022-04-04 15:32:00+00:00,Cere Network,0.036627,0.038402,0.036387,0.037804,6764200.95,59939283.17,1.585542e+09,0,0.057315,0,0,1
797,2022-04-03 00:00:00+00:00,2022-04-03 04:47:00+00:00,2022-04-03 21:49:00+00:00,Cere Network,0.037763,0.038201,0.036233,0.036640,6452226.63,58094209.23,1.585542e+09,0,0.086758,1,0,1
798,2022-04-02 00:00:00+00:00,2022-04-02 05:37:00+00:00,2022-04-02 15:55:00+00:00,Cere Network,0.038077,0.039603,0.036431,0.037759,7723944.93,59867811.88,1.585542e+09,0,0.043354,0,0,1


In [414]:
# Guardar los datos etiquetados
data.to_csv('datos_etiquetados_cere_network.csv', index=False)

#ENTRENAMIENTO

In [415]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Eliminar filas con valores NaN en la columna 'tags'
data_clean = data.dropna(subset=['tags'])

# Separar características (X) y etiquetas (y)
X = data_clean[['marketCap_state', 'high_volatility', 'supply_increase']]
y = data_clean['tags']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar y entrenar el modelo Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba con Random Forest
y_pred_rf = rf_model.predict(X_test)

# Evaluar el modelo Random Forest
print("Random Forest:")
print(classification_report(y_test, y_pred_rf))

# Inicializar y entrenar el modelo Gradient Boosting
gbm_model = GradientBoostingClassifier(n_estimators=100, random_state=42)
gbm_model.fit(X_train, y_train)

# Realizar predicciones en el conjunto de prueba con Gradient Boosting
y_pred_gbm = gbm_model.predict(X_test)

# Evaluar el modelo Gradient Boosting
print("\nGradient Boosting Machine:")
print(classification_report(y_test, y_pred_gbm))


Random Forest:
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       160

    accuracy                           1.00       160
   macro avg       1.00      1.00      1.00       160
weighted avg       1.00      1.00      1.00       160



ValueError: y contains 1 class after sample_weight trimmed classes with zero weights, while a minimum of 2 classes are required.

#EVALUACION

In [ ]:
import pandas as pd
import numpy as np

# Generar datos de prueba
num_samples = 917


# Etiquetas
test_marketCap_state = np.random.randint(0, 2, num_samples)  # 0 o 1 para el estado de capitalización de mercado
test_high_volatility = np.random.randint(0, 2, num_samples)  # 0 o 1 para alta volatilidad
test_supply_increase = np.random.randint(0, 2, num_samples)  # 0 o 1 para el aumento del suministro

# Crear el DataFrame de datos de prueba
test_data = pd.DataFrame({
    'marketCap_state': test_marketCap_state,
    'high_volatility': test_high_volatility,
    'supply_increase': test_supply_increase,

})

# Mostrar los primeros cinco registros del DataFrame de datos de prueba
print("Datos de prueba generados:")
print(test_data.head())


In [ ]:
# Predicciones con el modelo Random Forest
predicciones_rf_test = rf_model.predict(test_data)

# Predicciones con el modelo Gradient Boosting
predicciones_gbm_test = gbm_model.predict(test_data)

# Imprimir las predicciones
print("Predicciones Random Forest en datos de prueba:", predicciones_rf_test)
print("\nPredicciones Gradient Boosting en datos de prueba:", predicciones_gbm_test)


In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
# Etiquetar los datos originales con las mismas condiciones que usaste para etiquetar el conjunto de entrenamiento
y_true = data.apply(etiquetar_prediccion, axis=1)

# Mostrar las primeras cinco etiquetas verdaderas
print("Etiquetas verdaderas:")
print(y_true.head())

In [ ]:
test_data.shape

In [ ]:
y_true.shape

In [ ]:
# Calcular la precisión del modelo Random Forest en los datos de prueba
accuracy_rf = accuracy_score(y_true, predicciones_rf_test)
print("Precisión del modelo Random Forest en datos de prueba:", accuracy_rf)

# Calcular la precisión del modelo Gradient Boosting en los datos de prueba
accuracy_gbm = accuracy_score(y_true, predicciones_gbm_test)
print("Precisión del modelo Gradient Boosting en datos de prueba:", accuracy_gbm)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


In [ ]:
# Matriz de Confusión para el modelo Random Forest
confusion_matrix_rf = confusion_matrix(y_test, y_pred_rf)
plt.imshow(confusion_matrix_rf, cmap=plt.cm.Blues, interpolation='nearest')
plt.title('Matriz de Confusión - Random Forest')
plt.colorbar()
plt.xlabel('Etiquetas Predichas')
plt.ylabel('Etiquetas Verdaderas')
plt.xticks([0, 1, 2], ['Aumento', 'Disminución', 'Estable'])
plt.yticks([0, 1, 2], ['Aumento', 'Disminución', 'Estable'])
plt.show()


In [ ]:
# Obtener la importancia de las características del modelo Random Forest
importances = rf_model.feature_importances_

# Obtener nombres de las características
feature_names = X.columns

# Ordenar las importancias de las características en orden descendente
indices = np.argsort(importances)[::-1]

# Graficar la importancia de las características
plt.figure(figsize=(10, 6))
plt.title("Importancia de Características - Random Forest")
plt.bar(range(X.shape[1]), importances[indices], align="center")
plt.xticks(range(X.shape[1]), feature_names[indices], rotation=90)
plt.xlim([-1, X.shape[1]])
plt.xlabel('Características')
plt.ylabel('Importancia')
plt.show()


In [ ]:
from sklearn.tree import plot_tree

# Obtener un árbol de decisión del modelo Random Forest (o Gradient Boosting)
tree = rf_model.estimators_[0]  # Cambia 0 por el índice del árbol a visualizar

# Graficar el árbol de decisión
plt.figure(figsize=(20, 10))
plot_tree(tree, feature_names=X.columns, class_names=['Aumento', 'Disminución', 'Estable'], filled=True)
plt.show()
